# Title
#### June 29, 2019
* Flatiron School (nyc-mhtn-ds-0422019)

In [26]:
import json
import datetime

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)


from private import api_keys
from weather import Weather

#### Collision Data

In [27]:
# This file is saved locally from the Navy site because it is > 300 MB. 
# The Navy site is here: https://data.cityofnewyork.us/resource/qiz3-axqb.json

data_file = '../../Datasets/NYPD_Motor_Vehicle_Collisions.csv'
df = pd.read_csv(data_file)

## Data Cleaning
### Summarize the shape of collision data from NYC
`df.shape`
* (1507923, 29)
* 29 features (columns)
* 1,507,923 collisions (rows)



`df.info()`
* RangeIndex: 1507923 entries, 0 to 1507922
* Data columns (total 29 columns)
* dtypes: float64(4), int64(7), object(18)
* memory usage: 333.6+ MB

`DATE                             1507923 non-null object
TIME                             1507923 non-null object
BOROUGH                          1055432 non-null object
ZIP CODE                         1055253 non-null object
LATITUDE                         1319709 non-null float64
LONGITUDE                        1319709 non-null float64
LOCATION                         1319709 non-null object
ON STREET NAME                   1217162 non-null object
CROSS STREET NAME                1020629 non-null object
OFF STREET NAME                  198606 non-null object
NUMBER OF PERSONS INJURED        1507906 non-null float64
NUMBER OF PERSONS KILLED         1507892 non-null float64
NUMBER OF PEDESTRIANS INJURED    1507923 non-null int64
NUMBER OF PEDESTRIANS KILLED     1507923 non-null int64
NUMBER OF CYCLIST INJURED        1507923 non-null int64
NUMBER OF CYCLIST KILLED         1507923 non-null int64
NUMBER OF MOTORIST INJURED       1507923 non-null int64
NUMBER OF MOTORIST KILLED        1507923 non-null int64
CONTRIBUTING FACTOR VEHICLE 1    1504001 non-null object
CONTRIBUTING FACTOR VEHICLE 2    1306911 non-null object
CONTRIBUTING FACTOR VEHICLE 3    97164 non-null object
CONTRIBUTING FACTOR VEHICLE 4    20199 non-null object
CONTRIBUTING FACTOR VEHICLE 5    5142 non-null object
UNIQUE KEY                       1507923 non-null int64
VEHICLE TYPE CODE 1              1503118 non-null object
VEHICLE TYPE CODE 2              1263185 non-null object
VEHICLE TYPE CODE 3              126564 non-null object
VEHICLE TYPE CODE 4              48051 non-null object
VEHICLE TYPE CODE 5              10256 non-null object`



### Drop unnecessary columns
* 30334 rows × 29 columns do not have ANY location data

In [29]:
W = Weather()

dt = datetime('06/02/2019')
df.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,06/02/2019,0:00,BRONX,10469,40.877470,-73.842620,"(40.87747, -73.84262)",GUNTHER AVENUE,TILLOTSON AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Backing Unsafely,Unspecified,NaN,NaN,NaN,4143967,Sedan,Taxi,NaN,NaN,NaN
1,06/02/2019,0:00,BROOKLYN,11208,40.680016,-73.877430,"(40.680016, -73.87743)",ATLANTIC AVENUE,LOGAN STREET,NaN,0.0,0.0,0,0,0,0,0,0,Traffic Control Disregarded,Unspecified,NaN,NaN,NaN,4143956,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
2,06/02/2019,0:00,BROOKLYN,11215,40.671783,-73.974200,"(40.671783, -73.9742)",8 AVENUE,MONTGOMERY PLACE,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,4143697,Sedan,Sedan,NaN,NaN,NaN
3,06/02/2019,0:00,QUEENS,11432,40.707607,-73.787880,"(40.707607, -73.78788)",NaN,NaN,172-06 JAMAICA AVENUE,0.0,0.0,0,0,0,0,0,0,Passing or Lane Usage Improper,Unsafe Lane Changing,NaN,NaN,NaN,4143625,Sedan,Sedan,NaN,NaN,NaN
4,06/02/2019,0:00,NaN,NaN,40.627830,-74.169075,"(40.62783, -74.169075)",FOREST AVENUE,DWARF STREET,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4143471,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN


In [15]:
df['VEHICLE TYPE CODE 2'].unique()

array(['Taxi', 'Sedan', 'Station Wagon/Sport Utility Vehicle', nan,
       'Pick-up Truck', 'Carry All', 'Box Truck', 'Refrigerated Van',
       'Bike', 'Ambulance', 'MOPED', 'Bus', 'Convertible', 'AMBUL',
       'Motorscooter', 'Motorcycle', 'Van', 'Moped', 'Flat Bed',
       'Tractor Truck Diesel', 'Minibike', 'Multi-Wheeled Vehicle',
       'Lift Boom', 'Dump', 'Tanker', 'SCOOT', 'Garbage or Refuse',
       'Scoot', 'Chassis Cab', 'E BIK', 'Van Camper', 'Excav', 'motor',
       'van', 'CATER', 'Motorbike', 'Tractor Truck Gasoline',
       'Tow Truck / Wrecker', 'front', 'PK', 'Flat Rack', 'Armored Truck',
       'COM', 'SEMI-', 'BOX T', 'Beverage Truck', 'ambul', 'nyc b',
       'elect', 'Open Body', 'E Bik', 'Elect', 'ELECT', 'UNK', 'FDNY',
       'Stake or Rack', 'Concrete Mixer', 'CRWZK', 'P/SH', 'Ambul',
       'Pedicab', '3-Door', 'BULLD', 'garba', 'SANIT', 'FIRE', 'E-BIK',
       'Bulk Agriculture', 'DUMP', 'USPS', 'TRACT', 'TRUCK', 'ICE C',
       'BACKH', 'U-TRU', 'AMBU', 'T

In [8]:
df.loc[(df['LATITUDE'].isnull()) & (df['LONGITUDE'].isnull()) & (df['LOCATION'].isnull()) & (df['ON STREET NAME'].isnull()) & (df['OFF STREET NAME'].isnull()),]




,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
3385,05/28/2019,7:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4144598,Sedan,NaN,NaN,NaN,NaN
52893,03/04/2019,6:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4090802,Sedan,NaN,NaN,NaN,NaN
74825,01/25/2019,6:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4068736,Pick-up Truck,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
88338,12/30/2018,12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4054741,Sedan,Sedan,NaN,NaN,NaN
88339,12/30/2018,12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4054750,Sedan,Sedan,NaN,NaN,NaN
91050,12/25/2018,14:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4052066,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
92104,12/23/2018,16:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Following Too Closely,NaN,NaN,NaN,4051469,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
96563,12/17/2018,7:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4048628,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
106665,12/01/2018,11:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Passing or Lane Usage Improper,NaN,NaN,NaN,4036100,Sedan,Sedan,NaN,NaN,NaN
111641,11/23/2018,13:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unspecified,Unspecified,NaN,NaN,4031709,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN


In [ ]:
drop_columns = ['LOCATION', 'BOROUGH', 'LATITUDE', 'LONGITUDE', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'CONTRIBUTING FACTOR VEHICLE 1', 'ZIP CODE', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5', 'UNIQUE KEY', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4',
       'VEHICLE TYPE CODE 5']
df = df.drop(drop_columns, axis=1)

In [ ]:
df.isna().sum()

### NaNs
`DATE                              0
TIME                              0
NUMBER OF PERSONS INJURED        17
NUMBER OF PERSONS KILLED         31
NUMBER OF PEDESTRIANS INJURED     0
NUMBER OF PEDESTRIANS KILLED      0
NUMBER OF CYCLIST INJURED         0
NUMBER OF CYCLIST KILLED          0
NUMBER OF MOTORIST INJURED        0
NUMBER OF MOTORIST KILLED         0
dtype: int64
`
#### Drop NaNs
Relative to the total 1.5M collisions, the number of collisions for the two features with NaNs is low.  Let's just get rid of them.

In [ ]:
df = df.dropna()
df.isna().sum()

### Convert Data Types
#### Dates
(ATTENTION: the code below will take a few minutes)

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['TIME'] = pd.to_datetime(df['TIME'])

####  Floats

In [ ]:
df['NUMBER OF PERSONS INJURED'] = df['NUMBER OF PERSONS INJURED'].astype(int)
df['NUMBER OF PERSONS KILLED'] = df['NUMBER OF PERSONS KILLED'].astype(int)

### Saving data to file locally
The section below is commented out on purpose.  But if desired, can be used a breakpoint for saving cleaned data to save time in development.

In [ ]:
##############################
# BREAKPOINT — Save to disk (json)
# data_file = 'data/cleaned_traffic_data.json'
# df.to_json(data_file)


# Load from disk (json)
# If you just want to start with the clean data, start here.
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# import statsmodels.api as sm
# from statsmodels.formula.api import ols
# import json

# data_file = 'data/cleaned_traffic_data.json'
# with open(data_file) as json_file:  
#     data = json.load(json_file)  
#     df = pd.DataFrame(data)

## Data Engineering
### Visualize daily collision totals

In [ ]:
collisions_by_date = df['DATE'].value_counts().reset_index()
collisions_by_date.describe()

In [ ]:
plt.hist(collisions_by_date['DATE'], bins=50)
plt.xlabel('Collision Counts, bins=50')
plt.ylabel('Frequency (Days)')
plt.title('Histogram of Daily Collision Counts')
plt.savefig('data/histogram-collisions-per-day.png')

### Convert to days of the week

In [ ]:
weekdays = df['DATE'].value_counts().reset_index()
weekdays['day_of_week'] = weekdays['index'].dt.day_name()
weekdays.drop('index', axis=1, inplace=True)

In [ ]:
weekdays = weekdays.rename(index=str, columns={"DATE": "collisions"})

## Hypothesis Testing
### T2 Days of the Week (ANOVA)—Does the day of the weekday have an effect on the number of collisions?
* __Null Hypothesis (H<sub>0</sub>)__:  The day of a weekday does not have an effect on the number of collisions
* __Alternative Hypothesis (H<sub>1</sub>)__: The day of a weekday _does_ have an effect on the number of collisions

#### Box plots for each day of the week

In [ ]:
fig = plt.figure(figsize=(15,8))
sns.boxplot(x="day_of_week", y="collisions", data=weekdays, order=['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'])
plt.xlabel('')
plt.ylabel('Number of Collisions')
plt.title('NYC Collision Data from 2012–2019')
plt.savefig('images/boxplot-collisions-by-day.png')

#### ANOVA results from OLS

In [ ]:
formula = 'collisions ~ C(day_of_week)'
lm = ols(formula, weekdays).fit()
table = sm.stats.anova_lm(lm, typ=2)
table

### T3 Build Day Pair Boxplots
* Tuesday vs Thursday
* Saturday vs Sunday
* Friday vs Sunday

In [ ]:
def boxplot_daypairs(pairs, df):
    for pair in pairs:
        filename = 'images/boxplots/boxplot-collisions-by-' + pair[0] + '-' + pair[1] + '.png'
        pair_df = df[(df['day_of_week'] == pair[0]) | (df['day_of_week'] == pair[1])]
        fig = plt.figure(figsize=(15,8))
        sns.boxplot(x="day_of_week", y="collisions", data=pair_df)
        plt.xlabel('')
        plt.ylabel('Number of Collisions')
        plt.title('NYC Collision Data from 2012–2019')
        plt.savefig(filename)
    
    
day_pairs = [['Thursday', 'Tuesday'], ['Friday', 'Sunday'], ['Saturday', 'Sunday']]
boxplot_daypairs(day_pairs, weekdays)

### T3 Build Day Pair Historgrams and Calculate t-stat, p-value and Cohen's-d
* Tuesday vs Thursday
* Saturday vs Sunday
* Friday vs Sunday

In [ ]:
def Cohen_d(group1, group2):

    # Compute Cohen's d.

    # group1: Series or NumPy array
    # group2: Series or NumPy array

    # returns a floating point number 

    diff = group1.mean() - group2.mean()

    n1, n2 = len(group1), len(group2)
    var1 = group1.var()
    var2 = group2.var()

    # Calculate the pooled threshold as shown earlier
    pooled_var = (n1 * var1 + n2 * var2) / (n1 + n2)
    
    # Calculate Cohen's d statistic
    d = diff / np.sqrt(pooled_var)
    
    return d

def sample_variance(sample):
    sample_mean = np.mean(sample)
    return np.sum((sample - sample_mean) **2)/ (len(sample) -1)

def pooled_variance(sample1, sample2):
    n_1, n_2 = len(sample1), len(sample2)
    var_1, var_2 = sample_variance(sample1), sample_variance(sample2)
    return ((n_1-1) * var_1 + (n_2-1)* var_2)/((n_1 + n_2)-2)

def twosample_tstatistic(expr, ctrl):
    exp_mean, ctrl_mean = np.mean(expr), np.mean(ctrl)
    pool_var = pooled_variance(expr, ctrl)
    n_e, n_c = len(expr), len(ctrl)
    num = exp_mean - ctrl_mean
    denom = np.sqrt(pool_var * ((1/n_e)+(1/n_c)))
    return num / denom

def distplot_daypairs(pairs, df):
    for pair in pairs:
        filename = 'images/distplots/displot-collisions-by-' + pair[0] + '-' + pair[1] + '.png'
        day1 = df.loc[df['day_of_week'] == pair[0], 'collisions']
        day2 = df.loc[df['day_of_week'] == pair[1], 'collisions']
        t_stat = twosample_tstatistic(day1, day2)
        (stats_t, stats_p) = stats.ttest_ind(day1, day2)
        cd = Cohen_d(day1, day2)
        fig = plt.figure(figsize=(15,8))
        sns.set(rc={'figure.figsize':(12,10)})
        sns.distplot(day1)
        sns.distplot(day2)
        plt.xlabel('Collisions\nt-stat={}\nstats-t={}\nstats-p={}\ncohen_d={}'.format(t_stat, stats_t, stats_p, cd))
        plt.ylabel('')
        plt.title(pair[0] + ' (blue) and ' + pair[1] + ' (orange)')
        plt.savefig(filename)
    
    
day_pairs = [['Thursday', 'Tuesday'], ['Friday', 'Sunday'], ['Saturday', 'Sunday']]
distplot_daypairs(day_pairs, weekdays)

### T3 Visualize Effect
* Tuesday vs Thursday
* Saturday vs Sunday
* Friday vs Sunday

In [ ]:
def visualize_t(t_stat, n_control, n_experimental):

    # initialize a matplotlib "figure"
    fig = plt.figure(figsize=(8,5))
    ax = fig.gca()
    # generate points on the x axis between -4 and 4:
    xs = np.linspace(-4, 4, 500)

    # use stats.t.pdf to get values on the probability density function for the t-distribution
    
    ys= stats.t.pdf(xs, (n_control+n_experimental-2), 0, 1)
    ax.plot(xs, ys, linewidth=3, color='darkred')

    ax.axvline(t_stat, color='black', linestyle='--', lw=5)
#     ax.axvline(-t_stat, color='black', linestyle='--', lw=5)
    plt.show()
    return None

def visualize_t_daypairs(pairs, df):
    for pair in pairs:
        filename = 'images/distplots/displot-collisions-by-' + pair[0] + '-' + pair[1] + '.png'
        day1 = df.loc[df['day_of_week'] == pair[0], 'collisions']
        day2 = df.loc[df['day_of_week'] == pair[1], 'collisions']
        t_stat = twosample_tstatistic(day1, day2)
        visualize_t(t_stat, len(day1), len(day2))
    

visualize_t_daypairs(day_pairs, weekdays)

## T1 Moon Phases
The code below is somewhat redundant to the project because the original moon phase analysis was down outside this notebook, (see Top of this notebook).  However, we rebuilt the moonphase algorithms here in a parameterized way so that we could consider alternative distributions of moon phases.  

The original moon phase data came divided up into repeating four phases.  Counter to our anecdotal intuitions, each phase was defined by a moment in time _at its apex_.  We did not feel this capture the more _human_ notion of full moon, which gave as an offset +/- from the apex. 

This turned out to be quite complicated.

In any case, we tested three distributions of moon phase definitions:
* Even quarters with each phase starting at its apex
* Even quarters with the each phase starting at 45-degrees left of its apex
* Uneven quarters in which Full Moon and New Moon were roughly 1-day before and after its apex, and everthing else was a waxing or wane.



### Interpreting Moon Phase Data
####  Phase Interpretation 1: Full Moon and New Moon as notional "days", evetying else is waxing or waning

In [ ]:
labels = 'Full Moon', 'Waning', 'New Moon', 'Waxing'
sizes = [1, 5, 1, 5]
# explode = (0, 0.1, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()
# ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
#         shadow=True, startangle=90)
ax1.pie(sizes, labels=labels, startangle=75)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

####  Phase Interpretation 2: Even quarters with phase start at its apex (unintuitive)

In [ ]:
sizes = [3, 3, 3, 3,]
# explode = (0, 0.1, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()
# ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
#         shadow=True, startangle=90)
ax1.pie(sizes, labels=labels)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

####  Phase Interpretation 3: Even quarters with phase start at its apex (possibly most intuitive)

In [ ]:
sizes = [3, 3, 3, 3,]
# explode = (0, 0.1, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()
# ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
#         shadow=True, startangle=45)
ax1.pie(sizes, labels=labels, startangle=45)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

### Indexing Moon Phase Chronologically
It was important to build a model of moon phases that would accept an arbitrary date and return the appropriate phase.  This was not trivial.
#### Helper methods defined first

In [ ]:
filename = 'moonphasedf.csv'

def build_phases(filename):
    phases = {}
    f = open(filename)
    lines = f.readlines()
    f.close() 
    
    for i in range(1,len(lines)):
        try:
            date = datetime.datetime.strptime(lines[i].split(',')[1], '%Y %b %d')
            value = lines[i].split(',')[2]
        except Exception as e:
            print(e)
            continue

        if value not in phases:
            phases[value] = []
        phases[value].append(date)
        
    f.close()  
    
    return phases

def build_phases_from_string(filename):
    phases = {}
    f = open(filename)
    lines = f.readlines()
    f.close() 
    
    one_day = datetime.timedelta(days=0)
    six_days = datetime.timedelta(days=0)
    
    for i in range(1,len(lines)):
        try:
            date, value, time = lines[i].split(',')[1], lines[i].split(',')[2], lines[i].split(',')[3]
            yyyy, mmm, dd = date.split(' ')[0], date.split(' ')[1], date.split(' ')[2]
            hh, mm = time.split(':')[0], time.split(':')[1]
            mm = mm.strip()
            timestamp = datetime.datetime.strptime(date + ' ' + time, '%Y %b %d %H:%M\n')

        except Exception as e:
            print(e)
            continue
        
        adjusted_time = timestamp - one_day if ((value == 'Full Moon') | (value == 'New Moon')) else timestamp - six_days
        s = adjusted_time.strftime('%Y %b %d %H:%M\n')
#         s = timestamp.strftime('%Y %b %d %H:%M\n')
        yyyy, mmm, dd, time = s.split(' ')[0], s.split(' ')[1], s.split(' ')[2], s.split(' ')[3]
        hh, mm = time.split(':')[0], time.split(':')[1]
        mm = mm.strip()
        
        if yyyy not in phases:
            phases[yyyy] = {}
        if mmm not in phases[yyyy]:
            phases[yyyy][mmm] = {}
        if dd not in phases[yyyy][mmm]:
            phases[yyyy][mmm][dd] = {}
        if hh not in phases[yyyy][mmm][dd]:
            phases[yyyy][mmm][dd][hh] = {}
        
        value = 'Waxing' if value == 'First Quarter' else value
        value = 'Waning' if value == 'Last Quarter' else value
        phases[yyyy][mmm][dd][hh][mm] = value
        

    f.close()  
    
    return phases

def get_previous_phase(phase, direction):
    phase_strings = ['New Moon', 'Waxing', 'Full Moon', 'Waning']
    i = phase_strings.index(phase)
    return phase_strings[(i+direction)%4]

def get_phase_by_hour(dict_):
    minutes = list(dict_.keys())[0]
    return dict_[minutes]

def get_phase_by_day(dict_):
    hour = list(dict_.keys())[0]
    return get_phase_by_hour(dict_[hour])
    

def get_phase_from_day(day_string):
    try:
        s = day_string.strftime('%Y %b %d %H:%M\n')
        yyyy, mmm, dd, time = s.split(' ')[0], s.split(' ')[1], s.split(' ')[2], s.split(' ')[3]
        hh, mm = time.split(':')[0], time.split(':')[1]
        mm = mm.strip()
    except Exception as e:
        print(e)
        
    phase = 'No Phase'
    if yyyy in phases:
        if mmm in phases[yyyy]:
            if dd in phases[yyyy][mmm]:
                if hh in phases[yyyy][mmm][dd]:
                    if mm in phases[yyyy][mmm][dd][hh]:
                        phase = phases[yyyy][mmm][dd][hh][mm]
                    else:
                        minutes = list(phases[yyyy][mmm][dd][hh].keys())[0]
                        if int(mm) > int(minutes):
                            phase = phases[yyyy][mmm][dd][hh][minutes]
                        else:
                            phase = get_previous_phase(phases[yyyy][mmm][dd][hh][minutes], -1)
                else:
                    hours = list(phases[yyyy][mmm][dd].keys())[0]
                    phase = get_phase_by_hour(phases[yyyy][mmm][dd][hours])
                    if int(hh) < int(hours):
                        phase = get_previous_phase(phase, -1)
                    else:
                        print('hours', hh)
            else:
                days = list(phases[yyyy][mmm].keys())
                for day in days:
                    max_day = day
                    if int(day) > int(dd):
                        phase = get_phase_by_day(phases[yyyy][mmm][day])
                        phase = get_previous_phase(phase, -1)
                        break
                if int(dd) > int(max_day):
                    phase = get_phase_by_day(phases[yyyy][mmm][max_day])
        else:
            print('Month', mmm)
    else:
        print('year', yyyy)
                                    
    return phase



Now build the phase models

In [ ]:
phases = build_phases_from_string(filename) 

And plot the dataframe into a boxplot.

In [ ]:
moondays = df['DATE'].value_counts().reset_index()
moondays['phase'] = moondays['index'].map(lambda x: get_phase_from_day(x))
moondays.drop('index', axis=1, inplace=True)
moondays = moondays.rename(index=str, columns={"DATE": "collisions"})

In [ ]:
fig = plt.figure(figsize=(15,8))
sns.boxplot(x="phase", y="collisions", data=moondays, order=['Full Moon', 'Waning', 'New Moon', 'Waxing'])
plt.xlabel('')
plt.ylabel('Number of Collisions')
plt.title('NYC Collision Data from 2012–2019')
plt.savefig('images/boxplot-collisions-by-phase.png')

And check the ANOVA results from OLS

In [ ]:
formula = 'collisions ~ C(phase)'
lm = ols(formula, moondays).fit()
table = sm.stats.anova_lm(lm, typ=2)
table

And plot the distributions. Nearly all identical in any interpretation!

In [ ]:
def distplot_moonpairs(pairs, df):
    for pair in pairs:
        filename = 'images/distplots/displot-collisions-by-' + pair[0] + '-' + pair[1] + '.png'
        day1 = df.loc[df['phase'] == pair[0], 'collisions']
        day2 = df.loc[df['phase'] == pair[1], 'collisions']
        t_stat = twosample_tstatistic(day1, day2)
        (stats_t, stats_p) = stats.ttest_ind(day1, day2)
        cd = Cohen_d(day1, day2)
        fig = plt.figure(figsize=(15,8))
        sns.set(rc={'figure.figsize':(12,10)})
        sns.distplot(day1)
        sns.distplot(day2)
        plt.xlabel('Collisions\nt-stat={}\nstats-t={}\nstats-p={}\ncohen_d={}'.format(t_stat, stats_t, stats_p, cd))
        plt.ylabel('')
        plt.title(pair[0] + ' (blue) and ' + pair[1] + ' (orange)')
        plt.savefig(filename)
        plt.show()
    
    
day_pairs = [['Full Moon', 'Waxing'], ['Full Moon', 'Waning'], ['Full Moon', 'New Moon'], ['Waning', 'Waxing']]
distplot_moonpairs(day_pairs, moondays)

# We are trying to identify if there is a difference in NY mortality rate and national mortality rate in 2016

## H0: NYC mortality rate has no relationship to national mortality

## H1: NYC mortality rate has a relationship to national mortality rate

### Statistical test: Chi Square test of homogeneity. 

#### Formula: 
##### Degree of Freedom: 
DF = (r - 1) * (c - 1), where r is the number of populations, and c is the number of levels for the categorical variable.
##### Expected frequency counts. 
The expected frequency counts are computed separately for each population at each level of the categorical variable, according to the following formula: Er,c = (nr * nc) / n
##### Test statistic.
The test statistic is a chi-square random variable (Χ2) defined by the following equation : Χ2 = Σ [ (Or,c - Er,c)2 / Er,c ] where Or,c is the observed frequency count in population r for level c of the categorical variable, and Er,c is the expected frequency count in population r for level c of the categorical variable.

US Total Death: 34,439
US Total Injured: 2,177,000



ALPHA = 0.05

In [ ]:
import pandas as pd

In [ ]:
traffic_df = pd.read_csv('traffic__moonphase_df.csv')
traffic_df = traffic_df.drop('Unnamed: 0', axis = 1)

In [ ]:
traffic_12_17 = traffic_df.loc[ (traffic_df['DATE']>='2016-01-01') & (traffic_df['DATE']<='2016-12-31')]

In [ ]:
traffic_12_17.columns

In [ ]:
traffic_12_17.head()

In [ ]:
traffic_12_17.drop(['TIME','NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'MOONPHASE'],axis = 1, inplace = True)

In [ ]:
traffic_12_17.sum()

Calculating the sum of New York dead and injured.

In [ ]:
ny_dead = traffic_12_17['NUMBER OF PERSONS KILLED'].sum()

In [ ]:
ny_injured = traffic_12_17['NUMBER OF PERSONS INJURED'].sum()

In [ ]:
us_dead = 34439 #statistic from NHTSA
us_injured = 2177000

In [ ]:
degree_freedom = 1 #degree of freedom = (row-1)(column-1)

In [ ]:
total_dead= us_dead+ny_dead #calculating total
total_injured = us_injured+ny_injured
total_ny = ny_dead+ny_injured
total_us = us_dead+us_injured
total = total_ny+total_us

In [ ]:
p_dead =  total_dead/total #proportion of dead
p_dead

In [ ]:
p_injured= total_injured/total #proportion of injured
p_injured

In [ ]:
exp_dead_ny = total_ny*p_dead #calculating expected values 
exp_dead_ny
exp_injured_ny = total_ny*p_injured
exp_injured_ny

In [ ]:
exp_dead_us = total_us*p_dead
exp_injured_us = total_us*p_injured

In [ ]:
print(exp_dead_ny, exp_injured_ny, exp_dead_us, exp_injured_us)

In [ ]:
chi_square = ((ny_dead-exp_dead_ny)**2/exp_dead_ny 
              +(ny_injured-exp_injured_ny)**2/exp_injured_ny 
              + (us_dead-exp_dead_us)**2/exp_dead_us
              +(us_injured-exp_injured_us)**2/exp_injured_us)

In [ ]:
chi_square

In [ ]:
from scipy.stats.distributions import chi2
chi2.sf(chi_square,1)

## References
### Project Write-Up Template
Credit to dair.ai with ❤️
* https://github.com/dair-ai/writing_data_scientists/blob/master/Writing_Primer_for_Data_Scientists.ipynb